In [5]:
#Librerias que se utilizaran
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import duckdb
import glob

# Librerias astrofisica
# =============================================================================
from astroquery.mast import Observations
import lightkurve as lk
from astropy.time import Time

con = duckdb.connect(database=':memory:')

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
#Urls Pater
url_lc = "/home/pater/Desktop/CosasPater/UNAL/2023-02NovenoSemestre/Exoplanet_IA/Curvas_de_luz/"
url_lcc = "/home/pater/Desktop/CosasPater/UNAL/2023-02NovenoSemestre/Exoplanet_IA/Curvas_de_luz_cor/"
url    = "/home/pater/Desktop/CosasPater/UNAL/2023-02NovenoSemestre/Exoplanet_IA/"

In [6]:
#Url Ari 
url_lc = "C:/Users/ASUS/OneDrive/Documentos/2023-2/Inteligencia/Proyecto/Exoplanet_IA/Curvas_de_luz/"
url_lcc = "C:/Users/ASUS/OneDrive/Documentos/2023-2/Inteligencia/Proyecto/Exoplanet_IA/Curvas_de_luz_cor/"
url    = "C:/Users/ASUS/OneDrive/Documentos/2023-2/Inteligencia/Proyecto/Exoplanet_IA/"

## Conteo de estrellas por clase

In [15]:
Carac_Stars = pd.read_csv('Carac_Stars.csv')
Carac_Stars

,Kepler_ID,Metalicidad,Clasificacion,incert_clas
0,10797460,0.136,CANDIDATE,1.000
1,10811496,-0.127,FALSE POSITIVE,0.000
2,10848459,-0.522,FALSE POSITIVE,0.000
3,10854555,0.081,CANDIDATE,1.000
4,10872983,-0.067,CANDIDATE,0.992
...,...,...,...,...
7677,10128825,-0.046,CANDIDATE,0.497
7678,10147276,-0.038,FALSE POSITIVE,0.021
7679,10156110,0.082,FALSE POSITIVE,0.000
7680,8460634,-0.556,FALSE POSITIVE,0.000


In [17]:
Carac_Stars[Carac_Stars['Kepler_ID']==10004515]

,Kepler_ID,Metalicidad,Clasificacion,incert_clas
4962,10004515,-0.511,FALSE POSITIVE,NaN


In [4]:
con.execute("""select Clasificacion, count(*)
               from Carac_Stars
               group by Clasificacion""").df()

,Clasificacion,count_star()
0,CANDIDATE,3246
1,FALSE POSITIVE,4436


## Descripción de las series de tiempo

### Registros en el tiempo

In [7]:
#Ver una curva de luz
arch = os.listdir(url_lc)
serie = pd.read_csv(url_lc+arch[1])
serie

,time,flux,flux_err
0,1559.226355,NaN,NaN
1,1559.246789,51267.110,7.347812
2,1559.267223,51263.164,7.347730
3,1559.287658,51268.223,7.351565
4,1559.308092,51273.820,7.349178
...,...,...,...
1281,1590.919913,50892.176,7.319856
1282,1590.940347,50900.105,7.317449
1283,1590.960782,50887.600,7.318842
1284,1590.981216,50906.684,7.311092


In [8]:
#Los datos se encuentran en bkjd (Barycentric Kepler Julian Date)
time = Time(serie['time'], format='bkjd')  # Crear un objeto Time de astropy
datetime_time = time.datetime  # Convertir a datetime
help(datetime_time[0])

Help on datetime object:

class datetime(date)
 |  datetime(year, month, day[, hour[, minute[, second[, microsecond[,tzinfo]]]]])
 |  
 |  The year, month and day arguments are required. tzinfo may be None, or an
 |  instance of a tzinfo subclass. The remaining arguments may be ints.
 |  
 |  Method resolution order:
 |      datetime
 |      date
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __le__(self, value, /)
 |      Return self<=value.
 |  
 |  __lt__(self, value, /)
 |      Return self<value.
 |  
 |  __ne__(self, value, /)
 |      Return self!=value.
 |  
 |  __radd__(self, value, /)
 |      Retur

In [9]:
arch = os.listdir(url_lc)

for star_lc in arch:
    #Se pasa la curva de luz a un dataframe de pandas
    serie = pd.read_csv(url_lc+star_lc)
    
    #Se elimina la primera fila con valores NA
    serie.drop(0, axis=0, inplace=True)
    
    
    #Se transforman los tiempos a datatime
    time = Time(serie['time'], format='bkjd')
    serie['time'] = time.datetime 
    
    #Guardo la serie arreglada
    serie.to_csv(f'{url}Curvas_de_luz_cor/{star_lc}', index=False)
    
    